In [1]:
import ee
import pandas as pd
from tqdm import tqdm

ee.Initialize(project='pranavkoka123') 

geometry = ee.Geometry.Rectangle([78.97, 30.70, 79.20, 30.79])

In [2]:
def calculate_ndsi_and_confidence(image):
    green = image.select('SR_B3').rename('Green')
    swir = image.select('SR_B6').rename('SWIR')
    ndsi = green.subtract(swir).divide(green.add(swir)).rename('NDSI')
    qa_pixel = image.select('QA_PIXEL')
    snow_confidence = qa_pixel.rightShift(5).bitwiseAnd(1).rename('Snow_Confidence')
    cloud_confidence = qa_pixel.rightShift(3).bitwiseAnd(1).rename('Cloud_Confidence')

    return image.addBands([ndsi, green, swir, snow_confidence, cloud_confidence])

In [3]:
def calculate_pixel_counts(image):
    s0c0 = image.select('Snow_Confidence').eq(0).And(image.select('Cloud_Confidence').eq(0))
    s1c0 = image.select('Snow_Confidence').eq(1).And(image.select('Cloud_Confidence').eq(0))
    s0c1 = image.select('Snow_Confidence').eq(0).And(image.select('Cloud_Confidence').eq(1))
    s1c1 = image.select('Snow_Confidence').eq(1).And(image.select('Cloud_Confidence').eq(1))

    counts = {
        's0c0': int(s0c0.reduceRegion(ee.Reducer.sum(), geometry, scale=30).get('Snow_Confidence').getInfo() or 0),
        's1c0': int(s1c0.reduceRegion(ee.Reducer.sum(), geometry, scale=30).get('Snow_Confidence').getInfo() or 0),
        's0c1': int(s0c1.reduceRegion(ee.Reducer.sum(), geometry, scale=30).get('Snow_Confidence').getInfo() or 0),
        's1c1': int(s1c1.reduceRegion(ee.Reducer.sum(), geometry, scale=30).get('Snow_Confidence').getInfo() or 0)
    }

    return counts

In [ ]:
def process_monthly_data(year, month):
    start_date = ee.Date.fromYMD(year, month, 1)
    end_date = start_date.advance(1, 'month')

    image = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2').filterDate(start_date, end_date).filterDate(start_date, end_date).filterBounds(geometry).first()

    image_ndsi_conf = calculate_ndsi_and_confidence(image)

    pixel_counts = calculate_pixel_counts(image_ndsi_conf)

    return {
        'year': year,
        'month': month,
        's0c0': pixel_counts['s0c0'],
        's1c0': pixel_counts['s1c0'],
        's0c1': pixel_counts['s0c1'],
        's1c1': pixel_counts['s1c1']
    }

# Loop over the years and months to generate the dataframe
results = []
for year in tqdm(range(2013, 2024)):
    if year == 2013:
        for month in range(4,13):
            result = process_monthly_data(year, month)
            results.append(result)
    else:    
        for month in range(1, 13):
            result = process_monthly_data(year, month)
            results.append(result)

# Convert the results to a DataFrame
df = pd.DataFrame(results)

# Display the DataFrame
print(df)

In [5]:
def get_bands_data(year, month):
    start_date = ee.Date.fromYMD(year, month, 1)
    end_date = start_date.advance(1, 'month')
    image = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2').filterDate(start_date, end_date).filterDate(start_date, end_date).filterBounds(geometry).first()
    image_ndsi_conf = calculate_ndsi_and_confidence(image)

    data = image_ndsi_conf.reduceRegion(reducer=ee.Reducer.toList(), geometry=geometry, scale=30, maxPixels=1e8).getInfo()

    ndsi_values = data.get('NDSI', [])
    snow_confidence_values = data.get('Snow_Confidence', [])
    cloud_confidence_values = data.get('Cloud_Confidence', [])
    swir = data.get('SWIR', [])
    green = data.get('Green', [])

    # min_length = min(len(ndsi_values), len(swir), len(green), len(snow_confidence_values), len(cloud_confidence_values))
    
    # if min_length == 0:
    #     print(f"No data for year {year}, month {month}. Skipping.")
    #     return []

    month_data = []
    for i in range(len(ndsi_values)):
        month_data.append({
            'year': year,
            'month': month,
            'ndsi': ndsi_values[i],
            'swir': swir[i],
            'green': green[i],
            's1c0': int(snow_confidence_values[i] == 1 and cloud_confidence_values[i] == 0),
            's0c1': int(snow_confidence_values[i] == 0 and cloud_confidence_values[i] == 1),
            's0c0': int(snow_confidence_values[i] == 0 and cloud_confidence_values[i] == 0)
        })

    return month_data

results = []
for year in tqdm(range(2013, 2024)):
    if year == 2013:
        for month in range(4,13):
            result = get_bands_data(year, month)
            results.extend(result)
    else:    
        for month in range(1, 13):
            result = get_bands_data(year, month)
            results.extend(result)

df_bands = pd.DataFrame(results)

print(df_bands)

100%|██████████| 11/11 [24:17<00:00, 132.52s/it]


          year  month      ndsi   swir  green  s1c0  s0c1  s0c0
0         2013      4  0.605625   7951  32371     1     0     0
1         2013      4  0.588216   8121  31322     1     0     0
2         2013      4  0.598631   7976  31768     1     0     0
3         2013      4  0.607796   7919  32463     1     0     0
4         2013      4  0.606939   7987  32653     1     0     0
...        ...    ...       ...    ...    ...   ...   ...   ...
31275742  2023     12  0.183154  21064  30510     0     1     0
31275743  2023     12  0.175733  17622  25136     0     1     0
31275744  2023     12  0.196210  17943  26703     0     1     0
31275745  2023     12  0.197210  19108  28496     0     1     0
31275746  2023     12  0.187884  19572  28628     0     1     0

[31275747 rows x 8 columns]


In [6]:
df_bands['swir'] = df_bands['swir']*0.0000275 - 0.2
df_bands['green'] = df_bands['green']*0.0000275 - 0.2
df_bands['ndsi'] = (df_bands['green']-df_bands['swir'])/(df_bands['green']+df_bands['swir'])

In [7]:
df_bands.to_csv(r'data\bands_ndsi.csv')

In [ ]:
# year = 2014
# month = 5
# start_date = ee.Date.fromYMD(year, month, 1)
# end_date = start_date.advance(1, 'month')
# image = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2').filterDate(start_date, end_date).filterDate(start_date, end_date).filterBounds(geometry).first()
# image_ndsi_conf = calculate_ndsi_and_confidence(image)

# data = image_ndsi_conf.reduceRegion(reducer=ee.Reducer.toList(), geometry=geometry, scale=30, maxPixels=1e8).getInfo()

# ndsi_values = data.get('NDSI', [])
# snow_confidence_values = data.get('Snow_Confidence', [])
# cloud_confidence_values = data.get('Cloud_Confidence', [])
# swir = data.get('SWIR', [])
# green = data.get('Green', [])

In [ ]:
# print(len(ndsi_values), len(snow_confidence_values), len(cloud_confidence_values), len(swir), len(green))